Riding the Wave

Convection Problems

This module will explore the convection equation in more depth!



Conservation Laws

Conservation of Mass:

Rate of change of mass in the CV + rate of cahnge of mass across CS = 0

COM:

$$\frac{\partial}{\partial t} \int_{cv} \rho dV + \int_{cs} \rho u dA =0$$

Since CV is small we can take $\rho$ as uniform quantity inside it. 

$$\frac{\partial}{\partial t} \int_{cv} \rho dV > \frac{\partial \rho}{\partial t}dAdx$$